In [1]:
from pathlib import Path

import pandas as pd

In [2]:
dates_to_visualize = [
                                                                                            "2018-09", "2018-10", "2018-11", "2018-12",
    "2019-01", "2019-02", "2019-03", "2019-04", "2019-05", "2019-06", "2019-07", "2019-08", "2019-09", "2019-10", "2019-11", "2019-12", 
    "2020-01", "2020-02", "2020-03", "2020-04", "2020-05", "2020-06", "2020-07", "2020-08", "2020-09", "2020-10", "2020-11", "2020-12",
    "2021-01", "2021-02", "2021-03", "2021-04", "2021-05", "2021-06", "2021-07", "2021-08", "2021-09", "2021-10", "2021-11", "2021-12",
    "2022-01", "2022-02", "2022-03", "2022-04", "2022-05", "2022-06", "2022-07", "2022-08", "2022-09", "2022-10", "2022-11", "2022-12",
    "2023-01", "2023-02",
    # "2024-02",
]

In [3]:
available_data_files = list(Path("../data/httparchive_metrics/nel_config").glob("*.parquet"))

used_data_files = list(filter(lambda file: file.stem.split("_")[-1] in dates_to_visualize, available_data_files))
used_data_files

[WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2018-09.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2018-10.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2018-11.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2018-12.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2019-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2019-03.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2019-04.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2019-05.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2019-06.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2019-07.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2019-08.parquet'),
 WindowsPath('../data

### Success fraction configuration over months


In [4]:
# Pick out the Failure Fraction files
ff_files = list(filter(lambda file: file.stem[:file.stem.rfind("_")] == 'success_fraction', used_data_files))
ff_files

[WindowsPath('../data/httparchive_metrics/nel_config/success_fraction_2018-09.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/success_fraction_2018-10.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/success_fraction_2018-11.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/success_fraction_2018-12.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/success_fraction_2019-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/success_fraction_2019-03.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/success_fraction_2019-04.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/success_fraction_2019-05.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/success_fraction_2019-06.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/success_fraction_2019-07.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/success_fraction_2019-08.parquet'),
 WindowsPath('../data

### Aggregate results to visualize

In [5]:
from results.result_utils import concat_data_from_files

sf_data = concat_data_from_files(ff_files)
sf_data['nel_success_fraction'] = sf_data['nel_success_fraction'].astype('Float32')



sf_data

,date,nel_success_fraction,domain_count,domain_percent
0,2018-09,0.1,2,100.000000
0,2018-10,0.0,6,60.000000
1,2018-10,0.01,4,40.000000
0,2018-11,0.0,182,97.326203
1,2018-11,0.01,5,2.673797
...,...,...,...,...
9,2023-02,0.0001,18,0.000797
10,2023-02,0.25,2,0.000089
11,2023-02,0.02,9,0.000398
12,2023-02,0.5,8,0.000354


In [6]:
from results.result_utils import get_first_or_0, date_to_text_format

sf_result = pd.DataFrame({}, index=[
    '0.00',
    '0.00-0.01',
    '0.01',
    '0.01-0.05',
    '0.05',
    '0.05-0.10',
    '0.10',
    '0.10-0.25',
    '0.25',
    '0.25-0.50',
    '0.50',
    '0.50-1.00',
    '1.00',
])

for date in dates_to_visualize:
    month_sf_data = sf_data[sf_data['date'] == date]
    
    month_data_col = []
    
    # month_result['0.00']
    next_val = month_sf_data[month_sf_data['nel_success_fraction'] == 0].groupby(['date'])['domain_count'].sum()
    month_data_col.append(get_first_or_0(next_val))
    
    # month_result['0.00-0.01'] 
    next_val = month_sf_data[(month_sf_data['nel_success_fraction'] > 0) & (month_sf_data['nel_success_fraction'] < 0.01)].groupby(['date'])['domain_count'].sum()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['0.01']
    next_val = month_sf_data[month_sf_data['nel_success_fraction'] == 0.01].groupby(['date'])['domain_count'].sum()
    month_data_col.append(get_first_or_0(next_val))
    
    # month_result['0.01-0.05']
    next_val = month_sf_data[(month_sf_data['nel_success_fraction'] > 0.01) & (month_sf_data['nel_success_fraction'] < 0.05)].groupby(['date'])['domain_count'].sum()
    month_data_col.append(get_first_or_0(next_val))
    
    # month_result['0.05'] 
    next_val = month_sf_data[month_sf_data['nel_success_fraction'] == 0.05].groupby(['date'])['domain_count'].sum()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['0.05-0.10'] 
    next_val = month_sf_data[(month_sf_data['nel_success_fraction'] > 0.05) & (month_sf_data['nel_success_fraction'] < 0.10)].groupby(['date'])['domain_count'].sum()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['0.10']
    next_val = month_sf_data[month_sf_data['nel_success_fraction'] == 0.10].groupby(['date'])['domain_count'].sum()
    month_data_col.append(get_first_or_0(next_val))
    
    # month_result['0.10-0.25']
    next_val = month_sf_data[(month_sf_data['nel_success_fraction'] > 0.10) & (month_sf_data['nel_success_fraction'] < 0.25)].groupby(['date'])['domain_count'].sum()
    month_data_col.append(get_first_or_0(next_val))
    
    # month_result['0.25']
    next_val = month_sf_data[month_sf_data['nel_success_fraction'] == 0.25].groupby(['date'])['domain_count'].sum()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['0.25-0.50']
    next_val = month_sf_data[(month_sf_data['nel_success_fraction'] > 0.25) & (month_sf_data['nel_success_fraction'] < 0.50)].groupby(['date'])['domain_count'].sum()
    month_data_col.append(get_first_or_0(next_val))
    
    # month_result['0.50']
    next_val = month_sf_data[month_sf_data['nel_success_fraction'] == 0.50].groupby(['date'])['domain_count'].sum()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['0.50-1.00'] 
    next_val = month_sf_data[(month_sf_data['nel_success_fraction'] > 0.50) & (month_sf_data['nel_success_fraction'] < 1.00)].groupby(['date'])['domain_count'].sum()
    month_data_col.append(get_first_or_0(next_val))
    
    # month_result['1.00'] 
    next_val = month_sf_data[month_sf_data['nel_success_fraction'] == 1.00].groupby(['date'])['domain_count'].sum()
    month_data_col.append(get_first_or_0(next_val))
    
    sf_result[date_to_text_format(date)] = month_data_col
    
sf_result

,Sep 2018,Oct 2018,Nov 2018,Dec 2018,Jan 2019,Feb 2019,Mar 2019,Apr 2019,May 2019,Jun 2019,...,May 2022,Jun 2022,Jul 2022,Aug 2022,Sep 2022,Oct 2022,Nov 2022,Dec 2022,Jan 2023,Feb 2023
0.00,0,6,182,365,0,359,372,423,524,471,...,0,0,1141117,1552190,1526519,1619005,1709812,1704650,1657778,1631049
0.00-0.01,0,0,0,0,0,1,1,74156,80347,78734,...,0,0,815,7826,6217,7153,9485,10104,10220,9422
0.01,0,4,5,5,0,5,1,1,5,0,...,0,0,90982,524104,517177,552092,584021,599374,624056,617890
0.01-0.05,0,0,0,0,0,0,0,0,0,0,...,0,0,3,12,8,9,10,9,7,9
0.05,0,0,0,4,0,3,6,8,7,7,...,0,0,55,73,85,74,35,46,56,53
0.05-0.10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0.10,2,0,0,0,0,0,0,0,0,0,...,0,0,30,8,8,9,10,10,9,7
0.10-0.25,0,0,0,0,0,0,0,0,0,0,...,0,0,39,492,214,481,552,557,550,517
0.25,0,0,0,2,0,2,2,2,3,2,...,0,0,2,2,2,2,3,2,3,2
0.25-0.50,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
